In [1]:
def decorated_by(func):
    func.__doc__+='\nDcorated by decorated_by.'
    print(func.__doc__)
    return func

def add(x,y):
    """return the sum of x and y."""
    return x + y

add = decorated_by(add)#先运行decroated_by函数，返回一个add
print(add(3,4))

return the sum of x and y.
Dcorated by decorated_by.
7


In [3]:
def decorated_by_1(func):
    def pr(x,y):    #通过在函数中创建函数，让每次add调用都会输出print
        func.__doc__+='\nDcorated by decorated_by_1.'
        print(func.__doc__)
        return func(x,y)
    return pr

@decorated_by_1   #装饰器语法
def add(x,y):
    """return the sum of x and y."""
    return x+y

print(add(3,4)) #通过装饰器，函数实际上运行的是装饰后的状态
print(add(8,8))
print(add(10,10))  #函数也会创建一个自己的实例，这个实例中有内置属性，func.__doc__会记录每次调用add的状态，每次在print中输出。

return the sum of x and y.
Dcorated by decorated_by_1.
7
return the sum of x and y.
Dcorated by decorated_by_1.
Dcorated by decorated_by_1.
16
return the sum of x and y.
Dcorated by decorated_by_1.
Dcorated by decorated_by_1.
Dcorated by decorated_by_1.
20


In [ ]:
"""
使用装饰器最常见的理由：
在执行被装饰方法之前或之后添加额外的功能。
    包括检查身份、将函数结果记录到笃定位置等用例。
清理传递给函数的参数值，从而确保参数值得一致性或使该值符合特定的模式。
    清理函数的输出值，比如一个函数返回生成器，可以通过装饰器，返回一个Json或其他结构的数据。
函数注册，在任务运行器中注册一个任务。
"""

In [ ]:
"""
装饰器仅仅是这样的函数：（通常）接受被装饰的可调用函数作为唯一参数，
并且返回一个可调用函数。
一个很重要的注意事项：当装饰器应用到装饰函数时（而不是调用装饰器时），
会执行装饰代码本身。这一点可以从装饰器的本质中看到。
"""

In [ ]:
#建立一个注册类来实现函数顺序运行

In [6]:
class Registry:
    def __init__(self):
        self._functions = []
    
    def register(self,func):
        self._functions.append(func)
        return func
    
    def run_all(self,*args,**kwargs):
        for func in self._functions:
            func(*args,**kwargs)

In [14]:
a = Registry()
@a.register
def foo(n='foo'):
    print(n)
    
@a.register
def f1(n='f1'):
    print(n)
    
@a.register
def f2(n='f2'):
    print(n)
    
a.run_all()     #在函数被装饰的时候，就会加入Registry的实例a的内置属性_functions中，通过内置函数run_all()来运行

foo
f1
f2


In [ ]:
"""
几种装饰器的常用方法
添加额外功能
"""

In [32]:
#简单的类型检查器

def requires_ints(func):
    """
    用来检查函数接受的数据是否为int类型，
    如果是，则正常执行，否则抛出TypeError
    """
    def inner(*args,**kwargs):
        kwargs_values = [i for i in kwargs.values()]
        for arg in list(args) + kwargs_values:
            if not isinstance(arg,int):
                raise TypeError('%s only accepts integers as arguments.'%func.__name__)
        return func(*args,**kwargs)
    return inner

In [27]:
@requires_ints
def add(x,y):
    """
    返回两个int型数值的和
    """
    return x+y
add(5,3)

8

In [25]:
add(1.2,3)

TypeError: add only accepts integers as arguments.

In [35]:
#帮助保存信息
print(add.__doc__)   #被装饰器装饰的函数会失去很多基本的内置信息，但是这些信息可能会很重要，为了保存这些信息，可以使用wraps方法

None


In [36]:
from functools import wraps
def change_str_int(func):
    @wraps(func)#让func函数的内置属性被保存下来
    def inner(*args,**kwargs):
        kwargs_values = [ord(i) for i in kwargs.values()]
        in_args = (ord(i) for i in args)
        in_kwargs = dict(zip(kwargs.keys(),kwargs_values))
        return func(*in_args,**in_kwargs)
    return inner

In [46]:
@change_str_int
def add1(x,y):
    """
    接受一个字符作为参数，输出字符的ascII对应的值得和
    """
    return x + y

print(add1('a','b'))
print(add1.__name__,add1.__doc__)

195
add1 
    接受一个字符作为参数，输出字符的ascII对应的值得和
    


In [53]:
#用户验证：一种常见的用例（即在运行装饰方法之前执行某种正确性检查）是用户验证。

class User: #建立一个User数据类型，返回客户输入，查看是否已经注册，从而给予权限
    def __init__(self,username,email,password):
        self.username = username
        self.email = email
        self.password = password
    def __eq__(self,other):
        if self.username == other.username and self.email == other.email and self.password == other.password:
            return True
        else:
            return False
        
l = [User('wang','wang@126.com',123456),User('zhao','zhao@126.com',666666),User('li','li@126.com',111111)]#模仿注册数据

        
        
def requires_user(func): 
    """
    修饰器，用来验证客户是否注册，从而给予权限
    """
    @wraps(func)
    def inner(user,register=l,*args,**kwargs):
        """
        验证user中的数据是否在数据库中注册过，如果没有，则没有权限调用函数
        """
        if user and isinstance(user,User):
            if user in l:
                return func(*args,**kwargs)
            else :
                raise KeyError('你输入的信息有误')
        else:
            raise KeyError("输入的信息不完整，或者有误")
    return inner

In [54]:
@requires_user
def run_1(x):
    """
    需要首先判断运行权限的函数
    """
    print('执行了函数run_1,参数为%s'%x)

user = User('wang','wang@126.com',123456)
    
run_1(user,x=3) #这里参数必须用x=3的形式输入

执行了函数run_1,参数为3


In [56]:
run_1(User('li','li@126.com',232334))

KeyError: '你输入的信息有误'

In [ ]:
#格式化输出，格式化输出可以通过类中的内置函数__repr__或__str__方法，对于大量的相似输出，可以用装饰器

In [58]:
def print_doc(func):     #为函数添加特定的输出格式
    wraps(func)
    def inner(*args,**kwargs):
        print("{}函数的作用是：{}".format(func.__name__,func.__doc__))
        return func(*args,**kwargs)
    return inner

In [60]:
@print_doc         #一个简单的示例，对数据加工为特定的数据格式很有用，比如转换list结构到Dataframe，JSON，array等。
def run_2(*args):
    """
    输出所有参数的和
    """
    sum = 0
    for i in args:
        sum += i
    return sum

run_2(1,2,3,4,5)

run_2函数的作用是：
    输出所有参数的和
    


15

In [ ]:
#日志管理   

In [67]:
import time
from collections import namedtuple
logging = namedtuple('logging',['name','time'])

log = []
def logged(func):  #这个装饰器不会对函数产生什么影响，只是将执行函数的名称和时间加入log中
    wraps(func)
    def inner(*args,**kwargs):
        now = logging(func.__name__,time.ctime())
        log.append(now)
        return func(*args,**kwargs)
    return inner

In [68]:
@logged
def f1():
    print("执行f1")
    
@logged
def f2():
    print("执行f2")
    
@logged
def f3():
    print("执行f3")
    
f3()
f3()
f2()
f1()
f1()

执行f3
执行f3
执行f2
执行f1
执行f1


In [69]:
log

[logging(name='f3', time='Sun May 12 22:18:28 2019'),
 logging(name='f3', time='Sun May 12 22:18:28 2019'),
 logging(name='f2', time='Sun May 12 22:18:28 2019'),
 logging(name='f1', time='Sun May 12 22:18:28 2019'),
 logging(name='f1', time='Sun May 12 22:18:28 2019')]